In [189]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plot
import numpy as np
from numpy import genfromtxt
import csv
import sys, os, re
import pathlib

print(tf.__version__)

2.0.0


In [190]:
# The CANlab/WAVi Pain Study is Pre-Formatted to resemble BIDS neuroimaging formatting
# If your study does not abide to the following structure, please revisit previous scripts
# .../StudyRepo
# -------------> /raw
# -------------------> /*.art
# -------------------> /*.eeg
# -------------------> /*.evt
# -------------> /contigs
# -------------------> /train          2:1 train:test
# -------------------> /test

In [191]:
if "contigs" not in os.getcwd():
    try:
        os.chdir("CANlabStudy/contigs")
        directory=os.getcwd()
    except:
        print("I couldn't find the contigs folder.\n")
        directory = input("Please give the full path of the contigs folder: ")
        os.chdir(directory)

In [192]:
#train_path = pathlib.Path('train/p300/')
train_path = pathlib.Path('train/chronic/')
#train_path = pathlib.Path('train/')

# test_path = pathlib.Path('test/p300/')
test_path = pathlib.Path('test/chronic/')
#test_path = pathlib.Path('test/')

In [208]:
# load in image paths for training set

import random

train_image_paths = os.listdir(train_path)
#train_image_paths = list(train_path.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)

train_count = len(train_image_paths)
print("You have", train_count, "training images.")

# load in image paths for testing set?

test_image_paths = os.listdir(test_path)
#test_image_paths = list(test_path.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths if "ctrl" in path]
random.shuffle(test_image_paths)

test_count = len(test_image_paths)
print("You have", test_count, "testing images.")

You have 2580 training images.
You have 515 testing images.


In [209]:
#list the available labels
label_names = ['ctrl', 'pain']
#label_names = sorted(item.name for item in train_path.glob('*/') if item.is_dir())
print("Labels discovered:", label_names)

Labels discovered: ['ctrl', 'pain']


In [210]:
#assign an index to each label
label_to_index = dict((name, index) for index, name in enumerate(label_names))
print("Label indices:", label_to_index)

Label indices: {'ctrl': 0, 'pain': 1}


In [211]:
#create a list of every file and its index label

train_image_labels = [label_to_index[path[:4]] for path in train_image_paths]
#train_image_labels = [label_to_index[pathlib.Path(path).parent.name]
 #                   for path in train_image_paths]

test_image_labels = [label_to_index[path[:4]] for path in test_image_paths]    
#test_image_labels = [label_to_index[pathlib.Path(path).parent.name]
 #                   for path in test_image_paths]

In [212]:
def load_numpy_stack(lead, paths):
    numpy_stack = []
    for path in paths:
        path = lead+path
        array = genfromtxt(path, delimiter=",")
        array = array.reshape(array.shape +(1,))
        numpy_stack.append(array)
    return(numpy_stack)

In [213]:
train_arrays = load_numpy_stack('train/chronic/', train_image_paths)
#train_arrays = load_numpy_stack('', train_image_paths)

test_arrays = load_numpy_stack('test/chronic/', test_image_paths)
#test_arrays = load_numpy_stack('', test_image_paths)

In [214]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_arrays, train_image_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_arrays, test_image_labels))

In [215]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [216]:
def kerasModel(learn):
    model = tf.keras.Sequential([
        tf.keras.layers.Convolution2D(5, kernel_size=5, strides=5, padding="same", dilation_rate=1, activation="relu", data_format="channels_last", use_bias=False),
        tf.keras.layers.Convolution2D(5, kernel_size=5, strides=5, padding="same", dilation_rate=1, activation="relu", data_format="channels_last", use_bias=False),
        tf.keras.layers.Convolution2D(5, kernel_size=5, strides=5, padding="same", dilation_rate=1, activation="relu", data_format="channels_last", use_bias=False),
        #tf.keras.layers.Convolution2D(5, kernel_size=5, strides=5, padding="same", dilation_rate=1, activation="relu", data_format="channels_last", use_bias=False),
        tf.keras.layers.Dropout(0.5),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None),
        tf.keras.layers.Flatten(data_format="channels_last"),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learn),
                 loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])
    return(model)

In [217]:
def fitModel(model, epoch):
    model.fit(train_dataset, epochs=epoch)
    return(model)

In [218]:
def testModel(model):
    metrics = model.evaluate(test_dataset)
    return(metrics)

In [219]:
myresults = []

In [220]:
rates = np.arange(0.01, 0.1, 0.01)
beta1s = np.arange(0.9, 0.99, 0.01)
beta2s = np.arange(0.99, 0.999, 0.001)

epochs = int(input("How many epochs? "))
rate = float(input("What's my learning rate? "))

compiled = kerasModel(rate)
fitted = fitModel(compiled, epochs)
results = testModel(fitted)
myresults.append(results)

How many epochs? 100
What's my learning rate? 0.001

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/100
41/41 [==============================] - 1s 26ms/step - loss: 0.7284 - sparse_categorical_accuracy: 0.5698
Epoch 2/100
41/41 [==============================] - 1s 17ms/step - loss: 0.6754 - sparse_categorical_accuracy: 0.5775
Epoch 3/100
41/41 [==============================] - 1s 16ms/step - loss: 0.6709 - sparse_categorical_accuracy: 0.5984
Epoch 4/100
41/41 [==============================] - 1s 16ms/step - loss: 0.6687 - sparse_categoric

41/41 [==============================] - 1s 16ms/step - loss: 0.3711 - sparse_categorical_accuracy: 0.8314
Epoch 63/100
41/41 [==============================] - 1s 16ms/step - loss: 0.3636 - sparse_categorical_accuracy: 0.8411
Epoch 64/100
41/41 [==============================] - 1s 18ms/step - loss: 0.3724 - sparse_categorical_accuracy: 0.8306
Epoch 65/100
41/41 [==============================] - 1s 16ms/step - loss: 0.3716 - sparse_categorical_accuracy: 0.8322
Epoch 66/100
41/41 [==============================] - 1s 17ms/step - loss: 0.3592 - sparse_categorical_accuracy: 0.8322
Epoch 67/100
41/41 [==============================] - 1s 16ms/step - loss: 0.3685 - sparse_categorical_accuracy: 0.8364
Epoch 68/100
41/41 [==============================] - 1s 16ms/step - loss: 0.3714 - sparse_categorical_accuracy: 0.8372
Epoch 69/100
41/41 [==============================] - 1s 16ms/step - loss: 0.3705 - sparse_categorical_accuracy: 0.8380
Epoch 70/100
41/41 [==============================] -

In [117]:
myresults

# print("Loss: ", results[0], "\nAccuracy: ", results[1])

[[0.22637348339475435, 0.9122222], [0.3669460266828537, 0.8705556]]